In [ ]:
#Updated情感分析
'''
改进优化模型：涉及到如何使用压缩填充序列、加载和使用预训练词向量、采用不同的优化器、选择不同的RNN体系结构（包括双向RNN、多层RNN）和正则化。

本章主要内容如下：

序列填充
预训练词嵌入
LSTM
双向 RNN
多层 RNN
正则化
优化
'''


In [ ]:
#==1、准备数据==

# 设置seed，并将其分类训练、测试、验证集。

# 在准备数据的时候需要注意到，由于 RNN 只能处理序列中的非 padded 元素（即非0数据），对于任何 padded 元素输出都是 0 。所以注意到我们在准备数据的时候将include_length设置为True，以获得句子的实际长度，后续需要使用。
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  include_lengths = True)

LABEL = data.LabelField(dtype = torch.float)

In [ ]:
#2.1 加载 IMDb 数据集

from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

# 从训练集中选取部分做验证集

import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))



In [ ]:
# 2.2 词向量
'''

使用预训练词向量进行初始化操作，其中获取这些词向量是通过指定参数传递给 build_vocab 得到的。
选取GloVe词向量，GloVe的全称是：Global Vectors for Word Representation。
描述下如何使用此词向量,这里我们使用的是 "glove.6B.100d" ，其中，6B表示词向量是在60亿规模的tokens上训练得到的，100d表示词向量是100维的(注意,这个词向量有800多兆)

也可以选择其他的词向量。理论上，这些预训练词向量在词嵌入向量空间中的距离在一定程度上表征了词之间的语义关系，例如，“terrible”、“awful”、“dreadful” ，它们的词嵌入向量空间的距离会非常近。

TEXT.build_vocab表示从预训练的词向量中，将当前训练数据中的词汇的词向量抽取出来，构成当前训练集的 Vocab(词汇表)。对于当前词向量语料库中没有出现的单词（记为UNK，unknown），通过高斯分布随机初始化（unk_init = torch.Tensor.normal_）。

'''

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [ ]:
# 2.3 创建迭代器+选取GPU
BATCH_SIZE = 64

# 根据当前环境选择是否调用GPU进行训练
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 创建数据迭代器
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)


In [ ]:
# 2.4 构建模型

'''

使用 LSTM\双向RNN\多层RNN

正则化

当模型的参数逐渐增加的同时，模型过拟合的可能性就越大。为了解决这个问题，我们添加dropout正则化。Dropout的工作原理是在前向传播过程中，层中的神经元随机Dropout（设置为0）。每个神经元是否被drop的概率则由一个超参数设置，并不受其他神经元影响。

关于为什么dropout有效的一种理论是，参数dropout的模型可以被视为“weaker”（参数较少）的模型。因此，最终的模型可以被认为是所有这些weaker模型的集合，这些模型都没有过度参数化，因此降低了过拟合的可能性。

Implementation Details

1.针对模型训练过程中的一点补充：在模型训练过程中，对于每个样本中补齐后加上的pad token，模型是不应该对其进行训练的，也就是并不会学习“<pad>”标记的嵌入。因为padding token跟句子的情感是无关的。这就意味着pad token的嵌入层(词向量)会一直保持初始化的状态（初始化为全零）。具体而言，我们是通过往nn.Embedding 层传入pad token 的index索引，作为padding_idx参数。

2.因为实验中使用的双向LSTM的包含了前向传播和后向传播过程，所以最后的隐藏状态向量包含了前向和后向的隐藏状态，所以在下一层nn.Linear层中的输入的形状就是隐藏层维度形状的两倍。

3.在将embeddings(词向量)输入RNN前，我们需要借助nn.utils.rnn.packed_padded_sequence将它们‘打包’，以此来保证RNN只会处理不是pad的token。我们得到的输出包括packed_output (a packed sequence)以及hidden sate 和 cell state。如果没有进行‘打包’操作，那么输出的hidden state和cell state大概率是来自句子的pad token。如果使用packed padded sentences，输出的就会是最后一个非padded元素的hidden state 和 cell state。

4.之后我们借助nn.utils.rnn.pad_packed_sequence 将输出的句子‘解压’转换成一个tensor张量。需要注意的是来自padding tokens的输出是零张量，通常情况下，我们只有在后续的模型中使用输出时才需要‘解压’。虽然在本案例中下不需要，这里只是为展示其步骤。

5.final hidden sate：也就是hidden，其形状是[num layers * num directions, batch size, hid dim]。因为我们只要最后的前向和后向传播的hidden states,我们只要最后2个hidden layers就行hidden[-2,:,:] 和hidden[-1,:,:]，然后将他们合并在一起,再传入线性层linear layer。 #####这里不知道怎么解释会比较好，还需调整。

'''

import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        # embedding嵌入层（词向量）
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        # RNN变体——双向LSTM
        self.rnn = nn.LSTM(embedding_dim,  # input_size
                           hidden_dim,  #output_size
                           num_layers=n_layers,  # 层数
                           bidirectional=bidirectional, #是否双向
                           dropout=dropout) #随机去除神经元
        # 线性连接层
        self.fc = nn.Linear(hidden_dim * 2, output_dim) # 因为前向传播+后向传播有两个hidden sate,且合并在一起,所以乘以2
        
        # 随机去除神经元
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text 的形状 [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded 的形状 [sent len, batch size, emb dim]
        
        # pack sequence
        # lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output的形状[sent len, batch size, hid dim * num directions]
        #output中的 padding tokens是数值为0的张量
        
        #hidden 的形状 [num layers * num directions, batch size, hid dim]
        #cell 的形状 [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden 的形状 [batch size, hid dim * num directions]
            
        return self.fc(hidden)


In [ ]:
# 2.5 实例化模型+传入参数

# 为了保证pre-trained 词向量可以加载到模型中，EMBEDDING_DIM 必须等于预训练的GloVe词向量的大小。

INPUT_DIM = len(TEXT.vocab) # 250002: 之前设置的只取25000个最频繁的词,加上pad_token和unknown token
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] #指定参数,定义pad_token的index索引值,让模型不管pad token

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)
# 查看模型参数量


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

# 把前面加载好的预训练词向量复制进我们模型中的embedding嵌入层，用预训练的embeddings词向量替换掉原来模型初始化的权重参数。

# 我们从字段的vocab中检索嵌入，并检查它们的大小是否正确，[vocab size, embedding dim]

pretrained_embeddings = TEXT.vocab.vectors
# 检查词向量形状 [vocab size, embedding dim]
print(pretrained_embeddings.shape)

torch.Size([25002, 100])

# 用预训练的embedding词向量替换原始模型初始化的权重参数
model.embedding.weight.data.copy_(pretrained_embeddings)

#将unknown 和padding token设置为0
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

In [ ]:
# 2.6 训练模型

import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss() # 损失函数. criterion 在本task中时损失函数的意思

model = model.to(device)
criterion = criterion.to(device)

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad() # 梯度清零
        
        text, text_lengths = batch.text # batch.text返回的是一个元组(数字化的张量,每个句子的长度)  
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text  #batch.text返回的是一个元组(数字化的张量,每个句子的长度) 
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    # 保留最好的训练结果的那个模型参数，之后加载这个进行预测
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    

model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


In [ ]:
# 2.7 模型验证

import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()



In [ ]:
# 负面评论的例子：

predict_sentiment(model, "This film is terrible")
# 0.05380420759320259

#正面评论的例子：

predict_sentiment(model, "This film is great")
# 0.94941645860672

In [ ]:
# 现在已经为电影评论建立了一个情感分析模型。在下一小节中，我们将实现一个模型，这个模型会以更少的参数获得更高的精度、更快的训练速度。
# 参考资料
# https://blog.csdn.net/weixin_42167712/article/details/112196925